In [168]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
%matplotlib inline 
import matplotlib as plt

os.chdir("E:/data scientist/project edwiser/project 1")

In [178]:
testdata = pd.read_csv("test_data.csv")
traindata = pd.read_csv("train_data.csv")

In [179]:
mydata=traindata
mytest=testdata


In [180]:
#---------------------------------------------Exploratory Data Analysis------------------------------------------------------

# change type of varables in proper type
mydata['Churn']=mydata['Churn'].replace([" False."," True."],[0,1])
mydata['international plan']=mydata['international plan'].replace([" no"," yes"],[0,1])
mydata['voice mail plan']=mydata['voice mail plan'].replace([" no"," yes"],[0,1])

mytest['Churn']=mytest['Churn'].replace([" False."," True."],[0,1])
mytest['international plan']=mytest['international plan'].replace([" no"," yes"],[0,1])
mytest['voice mail plan']=mytest['voice mail plan'].replace([" no"," yes"],[0,1])

In [44]:
#--------------------------------------------Missing value Analysis--------------------------------------------------------
mydata.isnull().sum()
mytest.isnull().sum()
#there is no missing value present

state                            0
account length                   0
area code                        0
phone number                     0
international plan               0
voice mail plan                  0
number vmail messages            0
total day minutes                0
total day calls                  0
total day charge                 0
total eve minutes                0
total eve calls                  0
total eve charge                 0
total night minutes              0
total night calls                0
total night charge               0
total intl minutes               0
total intl calls                 0
total intl charge                0
number customer service calls    0
Churn                            0
dtype: int64

In [124]:
#----------------------------------------------------Outlier Analysis------------------------------------------------------
%matplotlib inline
#checking outliers with boxplot of numeric varables And replacing with max value
#storing numeric varables
allcol=mydata
#storing numerical varables only
allcol=allcol.drop(['state','international plan','voice mail plan','Churn','phone number'],axis=1) 
numer=allcol.columns  #storing column names of numerical varables
#creating loop that detect outlier and replace with max and min value present in particular varable
for i in numer:
    print(i)
    q75, q25= np.percentile(allcol.loc[:,i],[75,25])
    iqr=q75-q25
    min=q25-(1.5*iqr)
    max=q75+(1.5*iqr)
    print("min= ",min)
    print("max= ", max)
    mydata[mydata.loc[:,i]<min]=min
    mydata[mydata.loc[:,i]>max]=max


account length
min=  -5.5
max=  206.5
area code
min=  255.0
max=  663.0
number vmail messages
min=  -30.0
max=  50.0
total day minutes
min=  34.65
max=  325.45
total day calls
min=  46.5
max=  154.5
total day charge
min=  5.89
max=  55.33
total eve minutes
min=  63.55
max=  338.35
total eve calls
min=  46.5
max=  154.5
total eve charge
min=  5.4
max=  28.76
total night minutes
min=  64.55
max=  337.75
total night calls
min=  48.0
max=  152.0
total night charge
min=  2.915
max=  15.195
total intl minutes
min=  3.1
max=  17.5
total intl calls
min=  -1.5
max=  10.5
total intl charge
min=  0.845
max=  4.725
number customer service calls
min=  -0.5
max=  3.5


In [181]:
#---------------------------------------------Feature Selection---------------------------------------------------------------
import seaborn as sns
#setting height and width
#f,ax= plt.subplots(figsize=(7,5))
corr=allcol.corr()

#sns.heatmap(corr,mask=np.zero_like(corr,dtype=np.bool),cmap=sns.diverging_palette(200,10,as_cmap=True),square=True,ax=ax)

mydata=mydata.drop(['total day minutes','total night charge','total intl charge','total eve charge','phone number','state'],axis=1)
mytest=mytest.drop(['total day minutes','total night charge','total intl charge','total eve charge','phone number','state'],axis=1)

In [184]:
#**************************************************************************************************
#                                  Decision Tree model using C50
#***************************************************************************************************
x=mydata.values[:,0:14]
y=mydata.values[:,14]

#perform decision tree algo
clf=tree.DecisionTreeClassifier(criterion="entropy").fit(x,y)

pred_value=clf.predict(mytest.iloc[:,0:14])

#confusion matrix
conf_matrix=confusion_matrix(mytest['Churn'],pred_value)

conf_matrix_table=pd.crosstab(mytest['Churn'],pred_value)

#Saving true postive, true negtive,false postive, false negtive
true_neg=conf_matrix_table.iloc[0,0]
false_neg=conf_matrix_table.iloc[1,0]
true_post=conf_matrix_table.iloc[1,1]
false_post=conf_matrix_table.iloc[0,1]

#accuracy calculation

dTree_accu=accuracy_score(mytest['Churn'],pred_value)*100

dTree_accu=((true_post+true_neg)*100)/(true_neg+true_post+false_post+false_neg)

# false negtive rate calculation

false_negrate=(false_neg*100)/(false_neg+true_post)

# calculation of Recall

recall=(true_post*100)/(true_post+false_neg)


## Accuracy= 93.161    False Negtive Rate=29.464

In [185]:
# Random Forest Classifier

model_rf=RandomForestClassifier(n_estimators=2000).fit(x,y)

pred_value_rf=model_rf.predict(mytest.iloc[:,0:14])

#confusion matrix
conf_matrix_rf=confusion_matrix(mytest['Churn'],pred_value_rf)

conf_matrix_rftable=pd.crosstab(mytest['Churn'],pred_value_rf)

rftrue_neg=conf_matrix_rftable.iloc[0,0]
rffalse_neg=conf_matrix_rftable.iloc[1,0]
rftrue_post=conf_matrix_rftable.iloc[1,1]
rffalse_post=conf_matrix_rftable.iloc[0,1]

#accuracy calculation

#randamf_accu=accuracy_score(mytest['Churn'],pred_value_rf)*100

randam_accu=((rftrue_post+rftrue_neg)*100)/(rftrue_neg+rftrue_post+rffalse_post+rffalse_neg)

# false negtive rate calculation

false_neg_raterf=(rffalse_neg*100)/(rffalse_neg+rftrue_post)

# calculation of Recall

rf_recall=(rftrue_post*100)/(rftrue_post+rffalse_neg)


## Accuracy= 96.160    False Negtive Rate=26.339
